In [312]:
import os
from PIL import Image
import numpy as np

IMAGE_DIR = "images"
FOLDERS = [f for f in os.listdir(IMAGE_DIR) if "DS_Store" not in f]

In [314]:
images = []

for i, folder in enumerate(FOLDERS):
    fname = os.path.join(IMAGE_DIR, folder)
    for im in os.listdir(fname):
        impath = os.path.join(fname, im)
        img = Image.open(impath)
        data = np.asarray(img, dtype="int32")
        # Scale values from -1 to 1
        data = ((data / 255) - .5) / .5
        data = np.moveaxis(data, -1, 0)
        target = np.zeros((1, len(FOLDERS)))
        target[0,i] = 1
        images.append((data, target, folder, ))

In [304]:
def log_loss(predicted, actual):
    tol = 1e-6
    predicted = predicted
    actual = actual + tol
    return (np.log(predicted) - np.log(actual))

In [89]:
def softmax(preds):
    preds = np.exp(preds)
    return preds / np.sum(preds)

In [86]:
def init_layers(layer_defs):
    layers = []
    for i in range(1, len(layer_defs)):
        if "input_units" in layer_defs[i]:
            last_units = layer_defs[i]["input_units"]
        else:
            last_units = layer_defs[i-1]["units"]

        biases = np.ones((1,layer_defs[i]["units"]))
        if layer_defs[i]["type"] == "cnn":
            weights = np.random.rand(layer_defs[i-1]["units"], layer_defs[i]["units"], layer_defs[i]["kernel_size"], layer_defs[i]["kernel_size"])
        else:
            weights = np.random.rand(last_units, layer_defs[i]["units"])

        weights = weights / 5 - .1

        layers.append([
            weights,
            biases,
            layer_defs[i]["type"]
        ])
    return layers

In [110]:
import math

def unroll_image(image, kernel_x, kernel_y):
    x_size = (image.shape[0] - (kernel_x - 1))
    y_size = (image.shape[1] - (kernel_y - 1))
    rows =  x_size * y_size
    unrolled = np.zeros((rows, kernel_x * kernel_y))
    for x in range(0, x_size):
        for y in range(0, y_size):
            unrolled[y + (x * y_size),:] = image[x:(x+kernel_x),y:(y+kernel_y)].reshape((1,kernel_x * kernel_y))
    return unrolled

def convolve(image, kernel):
    return np.matmul(image, kernel.reshape(kernel.shape[0] * kernel.shape[1], 1))

In [190]:
def forward(batch, layers):
    hidden = [batch.copy()]
    for i in range(len(layers)):
        if layers[i][2] == "cnn":
            channels, next_channels, kernel_x, kernel_y = layers[i][0].shape

            new_x = batch.shape[1] - (kernel_x - 1)
            new_y = batch.shape[2] - (kernel_y - 1)
            next_batch = np.zeros((next_channels, new_x , new_y))
            for channel in range(channels):
                unrolled = unroll_image(batch[channel,:], kernel_x, kernel_y)
                for next_channel in range(next_channels):
                    kernel = layers[i][0][channel, next_channel, :]
                    mult = convolve(unrolled, kernel).reshape(new_x, new_y)
                    next_batch[next_channel,:] += mult
            next_batch /= batch.shape[0]

            hidden.append(next_batch.copy())
            next_batch = np.maximum(next_batch, 0)
            batch = next_batch
        else:
            if layers[i-1][2] == "cnn":
                batch = batch.reshape(batch.shape[0], batch.shape[1] * batch.shape[2])
            batch = np.matmul(batch, layers[i][0]) + layers[i][1]
            hidden.append(batch.copy())
            if i < len(layers) - 1:
                batch = np.maximum(batch, 0)

    return layers, batch, hidden

In [251]:
layer_defs = [
    {"type": "input", "units": 3},
    {"type": "cnn", "kernel_size": 3, "units": 1},
    {"type": "dense", "input_units": 254 * 254, "units": 5}
]

layers = init_layers(layer_defs)

In [252]:
layers, batch, hidden = forward(images[0][0], layers)
lr = 5e-4
grad = log_loss(softmax(batch), np.array([0,0,1,0,0]))

i = 1
grad = np.multiply(grad, np.heaviside(hidden[i+1], 1))

In [253]:
grad = grad.T
w_grad = np.matmul(grad, hidden[i].reshape(1, math.prod(hidden[i].shape))).T
b_grad = grad.T
layers[i][0] -= (w_grad + layers[i][0] * .01) * lr
layers[i][1] -= b_grad * lr
grad = np.matmul(layers[i][0], grad).T

In [254]:
grad.shape

(1, 64516)

In [255]:
i = 0
grad = grad.reshape(hidden[i+1].shape)
grad = np.multiply(grad, np.heaviside(hidden[i+1], 1))

In [256]:
_, kernel_x, kernel_y = grad.shape
flat_input = unroll_image(hidden[i][0,:], kernel_x, kernel_y)
flat_grad = grad.reshape(math.prod(grad.shape), 1)
k_grad = np.matmul(flat_input, flat_grad).reshape(1,1,layers[i][0].shape[2], layers[i][0].shape[3])
layers[i][0] -= k_grad * lr

In [260]:
k_grad.shape

(1, 1, 3, 3)

In [317]:
def backward(layers, hidden, grad, lr, verbose=False):
    for i in range(len(layers)-1, -1, -1):
        print(f"Layer {i}") if verbose else None

        if layers[i][2] == "cnn":
            grad = grad.reshape(hidden[i+1].shape)
            if i != len(layers) - 1:
                grad = np.multiply(grad, np.heaviside(hidden[i+1], 1))
            _, kernel_x, kernel_y = grad.shape
            for channel in range(hidden[i].shape[0]):
                flat_input = unroll_image(hidden[i][channel,:], kernel_x, kernel_y)
                flat_grad = grad.reshape(math.prod(grad.shape), 1)
                k_grad = np.matmul(flat_input, flat_grad).reshape(1, 1, layers[i][0].shape[2], layers[i][0].shape[3])
                print(f"k_grad: {k_grad.shape}") if verbose else None
                layers[i][0] -= k_grad * lr
        else:
            if i != len(layers) - 1:
                grad = np.multiply(grad, np.heaviside(hidden[i+1], 1))
            grad = grad.T
            print(f"starting grad: {grad.shape}") if verbose else None
            w_grad = np.matmul(grad, hidden[i].reshape(1, math.prod(hidden[i].shape))).T
            print(f"w_grad: {w_grad.shape}") if verbose else None
            b_grad = grad.T

            layers[i][0] -= (w_grad + layers[i][0] * .01) * lr
            layers[i][1] -= b_grad * lr

            grad = np.matmul(layers[i][0], grad).T
            print(f"ending grad: {grad.shape}") if verbose else None
    return layers

In [ ]:
layer_defs = [
    {"type": "input", "units": 3},
    {"type": "cnn", "kernel_size": 3, "units": 1},
    {"type": "dense", "input_units": 254 * 254, "units": 5}
]
lr = 5e-6
epochs = 1

layers = init_layers(layer_defs)
for epoch in range(epochs+1):
    epoch_loss = np.zeros(images[0][1].shape)
    for i, img in enumerate(images):
        image, target, label = img
        layers, batch, hidden = forward(image, layers)

        grad = log_loss(softmax(batch), target)
        epoch_loss += grad
        layers = backward(layers, hidden, grad, lr)
        if i % 50 == 0:
            print(f"Epoch {epoch} iter {i} loss: {np.mean(epoch_loss / i)}")

/var/folders/xz/9z84c__j28g8tg28bmcthjj00000gn/T/ipykernel_12135/759320627.py:20: RuntimeWarning: divide by zero encountered in divide
  print(f"Epoch {epoch} iter {i} loss: {np.mean(epoch_loss / i)}")


Epoch 0 iter 0 loss: nan
Epoch 0 iter 50 loss: 8.13878648512592
